# CIAN.ru

In [166]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from math import sin, cos, sqrt, radians

In [167]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

**Словарь, содержащий информацию о квартирах Москвы**

In [168]:
#Создание словаря с клчючом "District". По этому ключу будет хранится список,  
flatStats = {'District':[]} #инициализация списка, хранящего сведения о квартирах

**Введём численные обозначения округов Москвы**:
* 0 - ЦАО
* 1 - САО
* 2 - СВАО
* 3 - ВАО
* 4 - ЮВАО
* 5 - ЮАО
* 6 - ЮЗАО
* 7 - ЗАО
* 8 - СЗАО
* 9 - НАО
* 10 - ТАО

# Описание парсинга для ЦАО, для остальных округов действия будут аналогичными.
### Для ЦАО находим главную ссылку и в ней находим **&p=1&**, это и есть переход по страницам.

In [169]:
district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

In [170]:
district

'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

### Шагаем по страницам и качаем с них все ссылки на квартиры

In [171]:
links = []
for page in range(1, 30):
    page_url =  district.format(page)

    search_page = requests.get(page_url)
    search_page = search_page.content
    search_page = BeautifulSoup(search_page, 'lxml')

    flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
    flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

    for link in flat_urls:
        if link.isdigit():
            links.append(link)

In [172]:
links

['51730284',
 '149441292',
 '150882487',
 '149031259',
 '150346368',
 '149031321',
 '148794719',
 '150640194',
 '149849414',
 '149300855',
 '145609912',
 '149788538',
 '149727197',
 '149565205',
 '150463857',
 '148800467',
 '149031347',
 '148112883',
 '149031380',
 '149031376',
 '150459322',
 '150679023',
 '150678964',
 '150684424',
 '150720249',
 '147655377',
 '149031235',
 '150684509',
 '149896609',
 '150378355',
 '7368057',
 '150684571',
 '150627596',
 '150706578',
 '19220848',
 '150519279',
 '148829580',
 '149031293',
 '150378060',
 '150974485',
 '150545360',
 '149808503',
 '150645957',
 '150118457',
 '149031344',
 '150833962',
 '149031368',
 '149031262',
 '149390159',
 '149063476',
 '149507223',
 '149507258',
 '149031387',
 '149213362',
 '150945577',
 '146275883',
 '149172363',
 '150047932',
 '2929515',
 '149231113',
 '150043158',
 '149031278',
 '150638556',
 '147872339',
 '149431030',
 '150833977',
 '148677909',
 '150206839',
 '150405030',
 '149213364',
 '148024672',
 '50961693',

In [173]:
len(links)

812

Итого вытащили 812 квартир по ЦАО

### Переход на первую из них

In [174]:
flat_url = 'http://www.cian.ru/sale/flat/' + str(links[0]) + '/'
#flat_url = 'http://www.cian.ru/sale/flat/150531912/' 
flat_page = requests.get(flat_url)
flat_page = flat_page.content
flat_page = BeautifulSoup(flat_page, 'lxml')

In [175]:
flatStats['District']+=[0] #работаем с ЦАО

# Цена

In [176]:
price = flat_page.find('div', attrs={'class':'object_descr_price'})
price = re.split('<div>|за м<sup>2</sup>|\n|руб|\W', str(price))

In [177]:
print(price)

['', 'div', 'class', '', 'object_descr_price', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '129', '000', '000', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'div', '']


In [178]:
[i for i in price if i.isdigit()][-3:]

['129', '000', '000']

In [179]:
"".join([i for i in price if i.isdigit()][-3:])

'129000000'

In [180]:
price = "".join([i for i in price if i.isdigit()][-3:])

In [181]:
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = "".join([i for i in price if i.isdigit()][-3:])
    return int(price)

In [182]:
getPrice(flat_page)

129000000

In [183]:
flatStats['Price'] = [] #список под хранение цен на квартиры
flatStats['Price'] += [getPrice(flat_page)]

# Координаты местонахождения квартир

In [184]:
flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]

<a href="http://map.cian.ru/?deal_type=2&amp;flats=yes&amp;center=55.730426%2C37.574067&amp;room0=1&amp;room1=1&amp;room2=1&amp;room3=1" rel="nofollow" target="_blank"><i></i>Посмотреть предложения рядом</a>

In [185]:
coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
coords = re.split('&amp|center=|%2C', str(coords))
coords_list = []
for item in coords:
    if item[0].isdigit():
        coords_list.append(item)
lat = float(coords_list[0])
lon = float(coords_list[1])

In [186]:
lat, lon

(55.730426, 37.574067)

In [187]:
def getCoords(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0]) #широта
    lon = float(coords_list[1]) #долгота
    return lat, lon

In [188]:
getCoords(flat_page)

(55.730426, 37.574067)

# Расстояние от квартиры до центра Москвы(в км)

**Положим, что центром Москвы является Красная площадь**.

Её координаты:

широта: **55.753594**;

долгота: **37.622601**.

Так как площадь поверхности, занимаемой Москвой, мала по сравнению с площадью поверхности Земли, то при расчёте расстояний от квартиры до Красной площади кривизной поверхности можно пренебречь. Т.е. поверхность, занимаемая Москвой, -- это плоскость.


In [189]:
#Радиус Земли, в км 
R = 6371.0
#Географические координаты Красной Площади
lat_RS = 55.753594
lon_RS = 37.622601

Географические координаты тесно связаны со сферическими координатами, а сферические -- с декартовыми.

Переход от географических координат к декартовым координатам:

In [190]:
x = R*sin(radians(lat))*cos(radians(lon))
y = R*sin(radians(lat))*sin(radians(lon))
#координату z не рассматриваем в силу предположения о не кривизне поверхности  

In [191]:
#Декартовы координаты Красной площади
x_RS = R*sin(radians(lat_RS))*cos(radians(lon_RS))
y_RS = R*sin(radians(lat_RS))*sin(radians(lon_RS))

In [192]:
#искомое расстояние, в км
distance = sqrt(((x - x_RS)**2) + ((y - y_RS)**2))

In [193]:
distance

4.6902797956608495

In [194]:
def Dist(flat_page):
    #Радиус Земли, в км 
    R = 6371.0
    #Географические координаты Красной Площади
    lat_RS = 55.753594
    lon_RS = 37.622601
    
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0]) 
    lon = float(coords_list[1]) 
    
    x = R*sin(radians(lat))*cos(radians(lon))
    y = R*sin(radians(lat))*sin(radians(lon))
    x_RS = R*sin(radians(lat_RS))*cos(radians(lon_RS))
    y_RS = R*sin(radians(lat_RS))*sin(radians(lon_RS))
    
    return  sqrt(((x - x_RS)**2) + ((y - y_RS)**2))
    

In [195]:
Dist(flat_page)

4.6902797956608495

In [196]:
#Список под хранение расстояний до центра
flatStats['Dist'] = [] 
flatStats['Dist'] += [Dist(flat_page)]

# Число комнат

In [197]:
rooms = flat_page.find('div', attrs={'class':'object_descr_title'})

In [198]:
rooms

<div class="object_descr_title" itemprop="name">
                
    
        3-комн. кв. 
    

                
                    
                        в <a href="/zhiloy-kompleks-fyuzhn-park-moskva-5392/" target="_blank">ЖК «Фьюжн парк»</a>
</div>

In [199]:
rooms = html_stripper(rooms)

In [200]:
rooms

'\n                \n    \n        3-комн. кв.\xa0\n    \n\n                \n                    \n                        в\xa0ЖК «Фьюжн парк»\n'

In [201]:
room_number = ''
for i in re.split('-|\n', rooms):
    if 'комн' in i:
        break
    else:
        room_number += i
room_number = "".join(room_number.split())

In [202]:
room_number

'3'

In [203]:
def getRoom(flat_page):
    rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
    rooms = html_stripper(rooms)
    room_number = ''
    for i in re.split('-|\n', rooms):
        if 'комн' in i:
            break
        else:
            room_number += i
    room_number = "".join(room_number.split())
    try:
        return int(room_number)
    except:
        return '-' #в случае, если на странице не оказалась информация о числе комнат в квартире

In [204]:
getRoom(flat_page)

3

In [205]:
#Список под хранение числа комнат
flatStats['Rooms'] = [] 
flatStats['Rooms'] += [getRoom(flat_page)]

# Расстояние до метро в минутах

In [206]:
metrdist = flat_page.find('span', attrs={'class':'object_item_metro_comment'})

In [207]:
metrdist

<span class="object_item_metro_comment">
                8
                мин.
                
                    пешком
                
            </span>

In [208]:
metrdist = html_stripper(metrdist)

In [209]:
metrdist

'\n                8\n                мин.\n                \n                    пешком\n                \n            '

In [210]:
#разбиение строки metrdist
metrdist.split()

['8', 'мин.', 'пешком']

In [211]:
#Ищем расстояние до метро
for i in metrdist.split():
    if i.isdigit():
        print(int(i))

8


In [212]:
def Metrdist(flat_page):
    metrdist = flat_page.find('span', attrs={'class':'object_item_metro_comment'})
    metrdist = html_stripper(metrdist)
    for i in metrdist.split():
        if i.isdigit():
            return int(i)
        else:
            return '-' #в случае, если отсутствует информация о времени

In [213]:
Metrdist(flat_page)

8

In [214]:
#Список под хранение расстояния до метро в минутах.
flatStats['Metrdist'] = [] 
flatStats['Metrdist'] += [Metrdist(flat_page)]

# Расстояние до метро, если пешком

In [215]:
def Walk(flat_page):
    walk = flat_page.find('span', attrs={'class':'object_item_metro_comment'})
    walk = html_stripper(walk)
    if "пешком" in walk:
        return 1
    else:
        return 0

In [216]:
Walk(flat_page)

1

In [217]:
#Список под хранение результата предиката [расстояния до метро,если пешком].
flatStats['Walk'] = [] 
flatStats['Walk'] += [Walk(flat_page)]

# Общая информация о квартире

In [218]:
table = flat_page.find('table', attrs = {'class':'object_descr_props'})

In [219]:
table = html_stripper(table)

In [220]:
print(html_stripper(table)) #регулярные выражения для очистки от HTML-кода нам в помощь



 # Общая информация:


Этаж:

                5 / 11
                
            


Тип дома:

            вторичка
            
        


Тип продажи:
свободная


Общая площадь:

138,4 м2



Площадь комнат:

–
        


Жилая площадь:

–
        


Площадь кухни:

–
        


Санузел:
–
        


Балкон:
–
        


Лифт:
1 пасс. + 1 груз.


Вид из окна:
двор и улица




### Номер этажа, на котором расположена квартира

In [221]:
table.split()

['#',
 'Общая',
 'информация:',
 'Этаж:',
 '5',
 '/',
 '11',
 'Тип',
 'дома:',
 'вторичка',
 'Тип',
 'продажи:',
 'свободная',
 'Общая',
 'площадь:',
 '138,4',
 'м2',
 'Площадь',
 'комнат:',
 '–',
 'Жилая',
 'площадь:',
 '–',
 'Площадь',
 'кухни:',
 '–',
 'Санузел:',
 '–',
 'Балкон:',
 '–',
 'Лифт:',
 '1',
 'пасс.',
 '+',
 '1',
 'груз.',
 'Вид',
 'из',
 'окна:',
 'двор',
 'и',
 'улица']

In [222]:
#Ищем слово "Этаж" в общей информации о квартире
for i in range(len(table.split())):
    if "Этаж" in table.split()[i]: 
        print(table.split()[i+1]) 

5


In [223]:
#Объявление и определение функции, возвращающей номер этажа, на котором расположена квартира 
def getFloor(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    for i in range(len(table.split())):
        if "Этаж:" in table.split()[i]: 
            return int(table.split()[i+1])  #следующий элемент в списке после "Этаж" -- это номер этажа, на котором находится квартира
    
    #вдруг оказалось так, что нет информации об этаже
    if "Этаж:" not in table.split():
        return '-'

In [224]:
getFloor(flat_page)

5

In [225]:
#Список под хранение номера этажа, на котором расположена квартира
flatStats['Floor'] = []
flatStats['Floor'] +=[getFloor(flat_page)] 

### Всего этажей в доме

In [226]:
#Объявление и определение функции, возвращающей общее количество этажей в доме 
def getNfloors(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    for i in range(len(table.split())):
        if "Этаж:" in table.split()[i]: 
            try:
                return int(table.split()[i+3])  #третий элемент в списке после "Этаж" -- это общее количество этажей в доме
            except:
                return '-'
    #вдруг оказалось так, что нет информации об этаже или вообще отсутствует такое поле
    if "Этаж:" not in table.split():
        return '-'

In [227]:
getNfloors(flat_page)

11

In [228]:
#Список под хранение количество этажей в доме
flatStats['Nfloors'] = []
flatStats['Nfloors'] +=[getNfloors(flat_page)] 

### Тип дома: первичный рынок - 1,  вторичка - 0 

In [229]:
#Проверяем, в каком рынке находится дом
if "вторичка" in table:
    print(1)
else:
    print(0)

1


In [230]:
def is_New(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    if "вторичка" in table:
        return 1
    else:
        return 0

In [231]:
is_New(flat_page)

1

In [232]:
#Список под хранение информации о том, в каком рынке находится дом
flatStats['New'] = []
flatStats['New'] += [is_New(flat_page)]

### Балкон/лоджия

In [233]:
table.split()

['#',
 'Общая',
 'информация:',
 'Этаж:',
 '5',
 '/',
 '11',
 'Тип',
 'дома:',
 'вторичка',
 'Тип',
 'продажи:',
 'свободная',
 'Общая',
 'площадь:',
 '138,4',
 'м2',
 'Площадь',
 'комнат:',
 '–',
 'Жилая',
 'площадь:',
 '–',
 'Площадь',
 'кухни:',
 '–',
 'Санузел:',
 '–',
 'Балкон:',
 '–',
 'Лифт:',
 '1',
 'пасс.',
 '+',
 '1',
 'груз.',
 'Вид',
 'из',
 'окна:',
 'двор',
 'и',
 'улица']

In [234]:
#Функция, отвечающая на вопрос: есть ли в доме балкон/лоджия?
def is_there_Bal(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    for i in range(len(table.split())):
        if "Балкон:" in table.split()[i]: 
            try:
                int(table.split()[i+1])
                return 1
            except:
                return 0
    #вдруг оказалось так, что нет информации об этаже или вообще отсутствует такое поле
    if "Балкон:" not in table.split():
        return '-'

In [235]:
is_there_Bal(flat_page)

0

In [236]:
#Cписок под хранение информации о наличии балконов/лоджей  
flatStats['Bal'] = []
flatStats['Bal'] += [is_there_Bal(flat_page)]

### Наличие домашнего телефона в доме

In [237]:
def is_there_Tel(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    for i in range(len(table.split())):
        if "Телефон:" in table.split()[i]: 
            if table.split()[i+1]=='да':
                return 1
            else:
                return 0
    #вдруг оказалось так, что нет информации об этаже или вообще отсутствует такое поле
    if "Телефон:" not in table.split():
        return '-'

In [239]:
is_there_Tel(flat_page)

'-'

In [240]:
#Cписок под хранение информации о наличии телефона
flatStats['Tel'] = []
flatStats['Tel'] += [is_there_Tel(flat_page)]

### Тип дома

In [241]:
def is_Brick(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    if "кирпичный" in table:
        return 1
    elif "монолитный" in table:
        return 1
    elif "железобетонный" in table:
        return 1
    else:
        return 0 

In [242]:
is_Brick(flat_page)

0

In [243]:
#Cписок под хранение информации о типе дома
flatStats['Brick'] = []
flatStats['Brick'] += [is_Brick(flat_page)]

### Общая площадь квартиры

In [244]:
area_total_not_num = re.split('Общая площадь:|м2', table)[1]

In [245]:
#Преобразование общей площади к float
if ',' in area_total_not_num:
    area_total_not_num  = area_total_not_num.replace(',','.')
try:
    area_total = float(area_total_not_num)
except:
    area_total = '-'

In [246]:
area_total

138.4

In [247]:
def Totsp(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    
    area_total_not_num = re.split('Общая площадь:|м2', table)[1]

    if ',' in area_total_not_num:
        area_total_not_num  = area_total_not_num.replace(',','.')
    try:
        return float(area_total_not_num)
    except:
        return '-'  #вдруг, если отсутствует информация
 

In [248]:
Totsp(flat_page)

138.4

In [249]:
#Cписок под хранение общей площади
flatStats['Totsp'] = []
flatStats['Totsp'] += [Totsp(flat_page)]

### Жилая площадь квартиры

In [250]:
#Разбиение строки по 'Площадь', ':', 'м2'
re.split('Площадь|:|м2', table)

#Ищем "Жилая площадь" в разбиении, чтобы выйти на жилую площадь 
for i in range(len(re.split('Площадь|:|м2', table))):
    if "Жилая площадь" in re.split('Площадь|:|м2', table)[i]:
        area_live_not_num = re.split('Площадь|:|м2', table)[i+1] #жилая площадь в не числовом формате

In [251]:
area_live_not_num 

'\n\n–\n        \n\n\n'

In [252]:
#Преобразование жилой площади к float
if ',' in area_live_not_num:
    area_live_not_num = area_live_not_num.replace(',','.')
try:
    area_live = float(area_live_not_num)
except:
    area_live = '-'

In [253]:
area_live


'-'

In [254]:
#Автоматизируем приведённые выше действия
def Livesp(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    
    for i in range(len(re.split('Площадь|:|м2', table))):
        if "Жилая площадь" in re.split('Площадь|:|м2', table)[i]:
            area_live_not_num = re.split('Площадь|:|м2', table)[i+1]
    
    if ',' in area_live_not_num:
        area_live_not_num = area_live_not_num.replace(',','.')
    try:
        return float(area_live_not_num)
    except:
        return '-'

In [255]:
Livesp(flat_page)

'-'

In [256]:
#Cписок под хранение жилой площади
flatStats['Livesp'] = []
flatStats['Livesp'] += [Livesp(flat_page)]

### Площадь кухни 

In [257]:
#Разбиение строки по 'площадь', ':', 'м2'
fragmentation = re.split('Площадь|:|м2', table)

In [258]:
print(fragmentation)

['\n\n # Общая информация', '\n\n\nЭтаж', '\n\n                5\xa0/\xa011\n                \n            \n\n\nТип дома', '\n\n            вторичка\n            \n        \n\n\nТип продажи', '\nсвободная\n\n\nОбщая площадь', '\n\n138,4\xa0', '\n\n\n\n', ' комнат', '\n\n–\n        \n\n\nЖилая площадь', '\n\n–\n        \n\n\n', ' кухни', '\n\n–\n        \n\n\nСанузел', '\n–\n        \n\n\nБалкон', '\n–\n        \n\n\nЛифт', '\n1 пасс. + 1 груз.\n\n\nВид из окна', '\nдвор и улица\n\n']


In [259]:
#Ищем слово "кухни" в fragmentation, чтобы выйти на площадь кухни
for i in range(len(fragmentation)):
    if "кухни" in fragmentation[i]:
        area_kit_not_num = fragmentation[i+1] #площадь кухни в не числовом формате

In [260]:
area_kit_not_num

'\n\n–\n        \n\n\nСанузел'

In [261]:
#Преобразование площади кухни к float
if ',' in area_kit_not_num:
    area_kit_not_num = area_kit_not_num.replace(',','.')
try:
    area_kit = float(area_kit_not_num)
except:
    area_kit = '-'

In [262]:
area_kit

'-'

In [263]:
#функция, которая проделывает приведенные выше действия
def Kitsp(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    
    fragmentation = re.split('Площадь|:|м2', table)
    
    for i in range(len(fragmentation)):
        if "кухни" in fragmentation[i]:
            area_kit_not_num = fragmentation[i+1] #площадь кухни в не числовом формате
    
    if ',' in area_kit_not_num:
        area_kit_not_num = area_kit_not_num.replace(',','.')
    try:
        return float(area_kit_not_num)
    except:
        return '-'

In [264]:
 Kitsp(flat_page)

'-'

In [265]:
#Cписок под хранение площади кухни
flatStats['Kitsp'] = []
flatStats['Kitsp'] += [Kitsp(flat_page)]

# Данные о продаваемых квартирах (для одного окуруга)

## Функция как рабочий инструмент, который позволяет получать эти данные

In [283]:
#эта функция выполняет все приведенные выше действия для каждой из квартир, из каждого округа Москвы
def get_DATA_district(data,N_pages=30):
    #число рассматриваемых страниц с квартирами
    N_pages = 30
    
    #ЦАО
    if data == 'CAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 0
    
    #САО
    if data =='SAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=23&district%5B10%5D=33&district%5B11%5D=34&district%5B12%5D=35&district%5B13%5D=36&district%5B14%5D=37&district%5B15%5D=38&district%5B1%5D=24&district%5B2%5D=25&district%5B3%5D=26&district%5B4%5D=27&district%5B5%5D=28&district%5B6%5D=29&district%5B7%5D=30&district%5B8%5D=31&district%5B9%5D=32&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 1
    
    #СВАО
    if data == 'SVAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=39&district%5B10%5D=49&district%5B11%5D=50&district%5B12%5D=51&district%5B13%5D=52&district%5B14%5D=53&district%5B15%5D=54&district%5B16%5D=55&district%5B1%5D=40&district%5B2%5D=41&district%5B3%5D=42&district%5B4%5D=43&district%5B5%5D=44&district%5B6%5D=45&district%5B7%5D=46&district%5B8%5D=47&district%5B9%5D=48&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 2
    
    #ВАО
    if data == 'VAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B1%5D=57&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 3
    
    #ЮВАО
    if data == 'UVAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=72&district%5B10%5D=82&district%5B11%5D=83&district%5B1%5D=73&district%5B2%5D=74&district%5B3%5D=75&district%5B4%5D=76&district%5B5%5D=77&district%5B6%5D=78&district%5B7%5D=79&district%5B8%5D=80&district%5B9%5D=81&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 4
    
    #ЮАО
    if data == 'UAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=84&district%5B10%5D=94&district%5B11%5D=95&district%5B12%5D=96&district%5B13%5D=97&district%5B14%5D=98&district%5B15%5D=99&district%5B1%5D=85&district%5B2%5D=86&district%5B3%5D=87&district%5B4%5D=88&district%5B5%5D=89&district%5B6%5D=90&district%5B7%5D=91&district%5B8%5D=92&district%5B9%5D=93&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 5
        
    #ЮЗАО
    if data == 'UZAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=100&district%5B10%5D=110&district%5B11%5D=111&district%5B1%5D=101&district%5B2%5D=102&district%5B3%5D=103&district%5B4%5D=104&district%5B5%5D=105&district%5B6%5D=106&district%5B7%5D=107&district%5B8%5D=108&district%5B9%5D=109&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 6
    
    #ЗАО
    if data == 'ZAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=112&district%5B10%5D=122&district%5B11%5D=123&district%5B12%5D=124&district%5B13%5D=348&district%5B14%5D=349&district%5B15%5D=350&district%5B1%5D=113&district%5B2%5D=114&district%5B3%5D=115&district%5B4%5D=116&district%5B5%5D=117&district%5B6%5D=118&district%5B7%5D=119&district%5B8%5D=120&district%5B9%5D=121&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 7
   
    #СЗАО
    if data == 'SZAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=125&district%5B1%5D=126&district%5B2%5D=127&district%5B3%5D=128&district%5B4%5D=129&district%5B5%5D=130&district%5B6%5D=131&district%5B7%5D=132&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 8
    
    #НАО
    if data == 'NAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=327&district%5B10%5D=337&district%5B1%5D=328&district%5B2%5D=329&district%5B3%5D=330&district%5B4%5D=331&district%5B5%5D=332&district%5B6%5D=333&district%5B7%5D=334&district%5B8%5D=335&district%5B9%5D=336&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 9
    
    #ТАО
    if data == 'TAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=338&district%5B1%5D=339&district%5B2%5D=340&district%5B3%5D=341&district%5B4%5D=342&district%5B5%5D=343&district%5B6%5D=344&district%5B7%5D=345&district%5B8%5D=346&district%5B9%5D=347&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 10
        
    #Ссылки на квартиры
    links = []
    
    #Вытаскиваем ссылки на описание квартир
    for page in range(1, N_pages): #количество квартир по каждому округу пропорционально параметру N_pages
        page_url =  district.format(page)
        
        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')

        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))
        
        for link in flat_urls:
            if link.isdigit():
                links.append(link)
            
    flatStats['District']= []
    flatStats['Price'] = []
    flatStats['Rooms'] = []
    flatStats['Totsp'] = []
    flatStats['Kitsp'] = []
    flatStats['Livesp'] = []
    flatStats['Dist'] = [] 
    flatStats['Metrdist'] = [] 
    flatStats['Walk'] = []
    flatStats['Floor'] = [] 
    flatStats['Nfloors'] = [] 
    flatStats['New'] = []
    flatStats['Bal'] = []
    flatStats['Tel'] = []
    flatStats['Brick'] = []

    for i in range(len(links)):
        flat_url = 'http://www.cian.ru/sale/flat/' + str(links[i]) + '/'
        flat_page = requests.get(flat_url)
        flat_page = flat_page.content
        flat_page = BeautifulSoup(flat_page, 'lxml')
    
        flatStats['District'] += [district_encode]
        flatStats['Price'] += [getPrice(flat_page)]
        flatStats['Rooms'] += [getRoom(flat_page)]
        flatStats['Totsp'] += [Totsp(flat_page)]
        flatStats['Kitsp'] += [Kitsp(flat_page)]
        flatStats['Livesp'] += [Livesp(flat_page)]
        flatStats['Dist'] += [Dist(flat_page)] 
        flatStats['Metrdist'] += [Metrdist(flat_page)] 
        flatStats['Walk'] += [Walk(flat_page)]
        flatStats['Floor'] +=[getFloor(flat_page)] 
        flatStats['Nfloors'] +=[getNfloors(flat_page)] 
        flatStats['New'] += [is_New(flat_page)]
        flatStats['Bal'] += [is_there_Bal(flat_page)]
        flatStats['Tel'] += [is_there_Tel(flat_page)]
        flatStats['Brick'] += [is_Brick(flat_page)]
        
    #Данные возвращаются в формате DataFrame
    return pd.DataFrame(flatStats, columns=['District','Price','Rooms','Totsp','Livesp','Kitsp','Brick','Floor','Nfloors','Dist','Metrdist','Walk','Bal','Tel','New']) #в нумеровнии строк проблема

В этой функции аргумент data может принимать следующие значения:

In [284]:
data = ['CAO', 'SAO', 'SVAO', 'VAO', 'UVAO','UAO','UZAO','ZAO','SZAO','NAO','TAO'] # сокращённое названия округов на латинице

# Данные о продаваемых квартирах со всех округов Москвы 


In [285]:
#Выгрузка данных в формате DataFrame
INFO_about_selling_floats = pd.concat([get_DATA_district('CAO'), get_DATA_district('SAO'),get_DATA_district('SVAO'),get_DATA_district('VAO'),get_DATA_district('UVAO'),get_DATA_district('UAO'), get_DATA_district('UZAO'),get_DATA_district('ZAO'),get_DATA_district('SZAO'),get_DATA_district('NAO'), get_DATA_district('TAO')]) 

In [286]:
#Порядковые номера квартиры
N = pd.DataFrame(np.arange(1, len(INFO_about_selling_floats)+1), columns =['N'])
#Добавим в DataFrame порядковый номер квартиры
INFO_about_selling_floats = pd.concat([N,INFO_about_selling_floats],1)

# Наши данные о квартирах Москвы

In [287]:
INFO_about_selling_floats

,N,District,Price,Rooms,Totsp,Livesp,Kitsp,Brick,Floor,Nfloors,Dist,Metrdist,Walk,Bal,Tel,New
0,1,10,2580996,1,38.4,17.7,10.5,0,1,5,36.416321,35,0,0,-,0
1,2,10,3604279,2,56.4,36.5,7.8,0,2,5,36.416321,35,0,0,-,0
2,3,10,8500000,4,146.8,90.9,15.2,1,4,5,32.913593,-,0,1,-,1
3,4,10,1850000,1,33.4,17.6,7.9,1,1,3,66.664621,30,0,0,-,0
4,5,10,1900000,1,34.8,18.1,10.8,0,2,3,66.664621,-,0,0,-,0
5,6,10,2000000,1,35.0,18,10,1,3,3,66.664621,20,0,1,-,0
6,7,10,2070000,1,43.0,41.5,-,1,1,3,66.664621,30,0,1,-,0
7,8,10,2499000,1,42.0,19.4,11.6,1,3,4,61.057861,-,0,1,-,1
8,9,10,2608025,1,38.4,17.7,10.5,0,1,5,36.416321,35,0,0,-,0
9,10,10,2750000,2,56.2,55,-,1,1,3,66.664621,30,1,0,-,0


# Cохраняем данные в формате сsv

In [290]:
INFO_about_selling_floats.to_csv(r"D:\\CMF\Парсинг ЦИАНа\parsing.csv", index = False) #здесь Вы должны указать свой путь, куда Вы хотите сохранить файл 